In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.python import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Flatten,Conv2D,Dropout

In [15]:
path = './input/train.csv'
train = pd.read_csv(path)
test = pd.read_csv('./input/test.csv')
img_rows,img_cols = 28,28
num_classes = 10

In [16]:
def prep_data(raw):
    out_y = keras.utils.to_categorical(raw.label,num_classes)
    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images,img_rows,img_cols,1)
    out_x = x_shaped_array/255
    return out_x,out_y

In [17]:
X,y = prep_data(train)
model = Sequential()
model.add(Conv2D(20,kernel_size=(3,3),activation = 'relu',input_shape=(img_rows,img_cols,1)))
model.add(Dropout(0.5))
model.add(Conv2D(20,kernel_size=(3,3),activation='relu'))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])
model.fit(X,y,batch_size=128,epochs=2,validation_split=0.2)

Train on 33600 samples, validate on 8400 samples
Epoch 1/2
33600/33600 [==============================] - 3s 102us/step - loss: 0.3035 - acc: 0.9106 - val_loss: 0.0930 - val_acc: 0.9710
Epoch 2/2
33600/33600 [==============================] - 3s 85us/step - loss: 0.0877 - acc: 0.9735 - val_loss: 0.0707 - val_acc: 0.9756


In [19]:
def prep_data_test(raw):
    num_images = raw.shape[0]
    x_as_array = raw.values[:,:]
    x_shaped_array = x_as_array.reshape(num_images,img_rows,img_cols,1)
    out_x = x_shaped_array/255
    return out_x
X2 = prep_data_test(test)
model.fit(X,y,batch_size=128,epochs=2)

Epoch 1/2
42000/42000 [==============================] - 3s 81us/step - loss: 0.0586 - acc: 0.9818
Epoch 2/2
42000/42000 [==============================] - 3s 80us/step - loss: 0.0412 - acc: 0.9872


In [21]:
predictions = model.predict_classes(X2)
predictions_file = pd.DataFrame({
    'ImageId':list(range(1,len(predictions)+1)),
    'Label':predictions
})
predictions_file.to_csv('submission_MNIST.csv',index=False)